# 허깅페이스

[허깅 페이스(Hugging Face)](https://huggingface.co/)는 자연어 처리(NLP) 및 기계 학습 분야에 특화된 오픈소스 플랫폼이다. 이 플랫폼은 사전 학습된 모델, 데이터셋, 그리고 다양한 NLP 도구를 제공한다. [캐글(Kaggle)](https://www.kaggle.com/)이 주로 데이터 과학 경진대회와 다양한 도메인의 데이터셋에 중점을 두는 반면, Hugging Face는 NLP 기술의 개발과 공유에 초점을 맞춘다. Hugging Face의 'Model Hub'는 연구자와 개발자들이 자신의 모델을 쉽게 공유하고 다른 사람의 모델을 사용할 수 있게 해주며, 'Transformers' 라이브러리를 통해 최신 NLP 모델을 쉽게 구현하고 사용할 수 있다. 




```{mermaid}
graph LR
    A[허깅페이스]

    subgraph HF[모형과 데이터셋]
        B1[모형]
        B2[데이터셋]
        B3[파이프라인]
    end

    subgraph text[텍스트]
        C1[텍스트 <br> 감성 분류]
        C2[요약]
        C3[질의 응답]
    end

    subgraph image_audio[이미지와 오디오]
        D1[이미지<br>오디오 분류]
        D2[자동 음성 인식<br>STT]
    end

    subgraph advanced[고급 주제]
        E1[파인튜닝]
        E2[텍스트 생성<br>GPT]
        E3[임베딩<br>시맨틱 검색]
    end

    A --> HF
    A --> text
    A --> image_audio
    A --> advanced
```




## 설치

허깅페이스에서 모형(transformers)과 데이터셋(datasets)를 설치하면 다양한 모형과 데이터를 다운로드 받을 수 있다. 목적이 다양한 기계학습 모형을 개발하는 것이기 때문에 텍스트는 `torch`, 이미지 영상은 `torchvision` 오디오는 `torchaudio` 플레임워크를 설치한다.
`huggingface_hub`을 설치하면 CLI 방식으로 프로그래밍 방식으로 모형을 검색하고 설치에 도움을 받을 수 있다.




```{bash}
#| eval: false
#| label: hf-install
# 허깅 페이스
pip install transformers datasets
# ML 플레임워크
pip install torch torchvision torchaudio
## HF 허브
pip install huggingface_hub
```




### 모형

`huggingface_hub`으로 작업(task)에 맞는 모형을 `HfAPi` `list_models()` 메쏘드를 통해 찾아낼 수 있다.


In [ ]:
#| label: search-models
from huggingface_hub import HfApi
from pprint import pprint

api = HfApi()

models = api.list_models(
    task="table-question-answering", 
    sort="downloads", 
    direction=-1, 
    limit=1
)


def format_model_info(model):
    return {
        "id": model.id,
        "downloads": model.downloads,
        "likes": model.likes,
        "task": model.pipeline_tag,
        "last_modified": (
            model.last_modified.strftime("%Y-%m-%d %H:%M:%S")
            if model.last_modified
            else "N/A"
        ),
    }

formatted_models = [format_model_info(model) for model in models]

print("Table Question Answering Models:")
pprint(formatted_models, width=100, sort_dicts=False)

가장 많은 조회수와 다운로드를 자랑하는 텍스트 분류 작업(text-classificatin)에서 
[distilbert/distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english) 모형을 찾았으면 로컬 컴퓨터에 저장하여 다음 후속작업을 이어나갈 수 있다. 


In [ ]:
#| label: hf-download-model
#| eval: false
# Import AutoModel
from transformers import AutoModel

modelId = "distilbert-base-uncased-finetuned-sst-2-english"
# Download model using the modelId
model = AutoModel.from_pretrained(modelId)
# Save the model to a local directory
model.save_pretrained(save_directory=f"d:/llms/{modelId}")

`tokenizer_config.json`, `config.json`, `model.safetensors` 파일이 모두 다운로드 되었는지 확인한다.


In [ ]:
#| label: hf-model-helloworld
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import warnings

# 경고 필터 설정
warnings.filterwarnings("ignore", message="Some weights of the model checkpoint")

# 모델 ID와 저장 경로
model_id = "distilbert-base-uncased-finetuned-sst-2-english"
model_path = f"d:/llms/{model_id}"

# 모델 로드
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# 토크나이저 로드 (온라인에서 직접 다운로드)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 분석할 텍스트
text = "오늘 정말 기분이 좋다."

# 텍스트를 토큰화하고 모델 입력으로 준비
inputs = tokenizer(text, return_tensors="pt")

# 모델 추론
with torch.no_grad():
    outputs = model(**inputs)

# 결과 해석
probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
positive_probability = probabilities[0][1].item()

print(f"입력 텍스트: '{text}'")
print(f"긍정적인 감성일 확률: {positive_probability:.2%}")

if positive_probability > 0.5:
    print("결과: 긍정적인 감성입니다.")
else:
    print("결과: 부정적인 감성입니다.")

## 데이터셋

허깅페이스 데이터셋은 `3D`, `Audio`, `Geospatial`, `Image`, `Tabular`, `Text`, `Time-series`, `Video` 범주로 나누어져 제공되고 있다.
정형데이터(Tabular) 중에서 [lavita/medical-qa-shared-task-v1-toy](https://huggingface.co/datasets/lavita/medical-qa-shared-task-v1-toy) 데이터셋이 가장 많은 다운로드를 기록하고 있다.[^hf-dataset]
정형데이터를 판다스 데이터프레임으로 변환한 후 후속 작업을 진행한다.

[^hf-dataset]: [Working with Hugging Face Datasets](https://medium.com/towards-data-science/working-with-hugging-face-datasets-bba14dd8da68)


In [ ]:
from datasets import load_dataset

dataset = load_dataset("lavita/medical-qa-shared-task-v1-toy", split="train")

dataset.to_pandas().head()[["id", "ending0", "GCC"]]

## 파이프라인

**Auto 클래스**는 모델, 토크나이저, 설정 등을 유연하게 사용할 수 있는 일반 클래스로, ML 작업에 대한 더 많은 제어와 유연성을 제공한다. 
**AutoModels**는 특정 작업에 맞는 사전 훈련된 모델을 직접 다운로드하고 초기화하는 기능을 제공하여 복잡한 모델 아키텍처를 직접 구현할 필요 없이 바로 사용할 수 있다.
**AutoTokenizers**는 텍스트 입력 데이터를 모델이 이해할 수 있는 형식을 담당한다. 모델과 쌍을 이루는 토크나이저를 사용하는 것이 권장되며, 이를 통해 텍스트 전처리 과정을 자동화할 수 있다.

파이프라인 모듈은 특정 작업에 필요한 모든 단계를 포함하는 고수준 인터페이스로 모델 로딩, 전처리, 추론 등의 과정을 자동화하여 빠르게 작업을 수행하고 시작하기에 적합하다.





```{mermaid}
graph LR
    subgraph Auto클래스
        B[Auto 클래스]
        B --> C[AutoModels]
        B --> D[AutoTokenizers]
        B --> E[Auto 컴포넌트<br>* AutoConfig<br>* AutoProcessor<br>* AutoFeatureExtractor]
        
    end

    subgraph 파이프라인모듈
        F[파이프라인 모듈]
        F --> F2[모델 로딩]
        F --> F3[전처리]
        F --> F4[추론]
    end

    Auto클래스 --> 파이프라인모듈
```





::: panel-tabset

### 감성분석


In [ ]:
from transformers import pipeline

# 다국어 BERT 모델을 사용한 감성 분석 파이프라인 생성
classifier = pipeline(
    task="sentiment-analysis",
    model="bert-base-multilingual-cased",
    return_all_scores=True,
)

# 분석할 텍스트 리스트
texts = [
    "이 영화는 정말 재미있었어요. 추천합니다!",
    "서비스가 너무 별로였어요. 다시는 안 갈 것 같아요.",
    "그저 그랬어요. 특별히 좋지도 나쁘지도 않았습니다.",
    "This movie was really great. I recommend it!",
    "The service was terrible. I won't go there again.",
]

# 각 텍스트에 대해 감성 분석 수행
for text in texts:
    result = classifier(text)[0]

    # 긍정, 부정 점수 추출
    positive_score = next(score for score in result if score["label"] == "LABEL_1")[
        "score"
    ]
    negative_score = next(score for score in result if score["label"] == "LABEL_0")[
        "score"
    ]

    print(f"\n텍스트: {text}")
    print(f"긍정 점수: {positive_score:.2%}")
    print(f"부정 점수: {negative_score:.2%}")

    # 감성 판단
    if positive_score > negative_score:
        sentiment = "긍정"
        score = positive_score
    else:
        sentiment = "부정"
        score = negative_score

    print(f"판단된 감성: {sentiment}")

    # 확신도에 따른 해석
    if score > 0.75:
        interpretation = "매우 강한"
    elif score > 0.6:
        interpretation = "강한"
    elif score > 0.4:
        interpretation = "약한"
    else:
        interpretation = "중립적인"

    print(f"해석: {interpretation} {sentiment}적 의견입니다.")


### 텍스트 분류


In [ ]:
from transformers import pipeline

# 파이프라인 생성
classifier = pipeline(task="zero-shot-classification", model="facebook/bart-large-mnli")

# 예제 1: 영화 리뷰 분류
text1 = "이 영화는 흥미진진한 플롯과 멋진 특수효과로 가득했습니다. 배우들의 연기도 훌륭했어요."
candidate_labels1 = ["긍정적", "부정적", "중립적"]
output1 = classifier(text1, candidate_labels1)
print("영화 리뷰 분류:")
print(f"가장 높은 확률의 레이블: {output1['labels'][0]}")
print(f"확률: {output1['scores'][0]:.4f}")

# 예제 2: 뉴스 기사 주제 분류
text2 = "최근 연구에 따르면 규칙적인 운동이 스트레스 감소와 수면의 질 향상에 도움이 된다고 합니다."
candidate_labels2 = ["건강", "스포츠", "기술", "정치"]
output2 = classifier(text2, candidate_labels2)
print("\n뉴스 기사 주제 분류:")
print(f"가장 높은 확률의 레이블: {output2['labels'][0]}")
print(f"확률: {output2['scores'][0]:.4f}")

# 예제 3: 상품 리뷰 감성 분석
text3 = "이 제품은 가격은 비싸지만 품질이 정말 뛰어나고 오래 사용할 수 있을 것 같아요."
candidate_labels3 = ["만족", "불만족", "중립"]
output3 = classifier(text3, candidate_labels3)
print("\n상품 리뷰 감성 분석:")
print(f"가장 높은 확률의 레이블: {output3['labels'][0]}")
print(f"확률: {output3['scores'][0]:.4f}")

### STT

ASR(Automatic Speech Recognition)은 음성을 인식하여 텍스트로 출력하는 기계학습 모형으로 
OpenAI 위스퍼, [페이스북 Wav2Vec2](https://huggingface.co/facebook/wav2vec2-base-960h)이 유명하다.


In [ ]:
#| eval: false
from transformers import pipeline

# 한국어 음성 인식을 위한 모델 로드
transcriber = pipeline(
    task="automatic-speech-recognition", model="kresnik/wav2vec2-large-xlsr-korean"
)

# 오디오 파일 경로
audio_file = "data/한국어_영어번역.mp3"

# 음성 인식 실행
result = transcriber(audio_file)

print(result["text"])

```
정교는 국미럽 얼는 여던 레번째 사미지를 맞아 나라 리 기생하고 신하신 애국 습예들개 단았는 감사와 경위를 표합니또립 유공자와 유가적 여러분에게도 존경과 감사에 말씀을 쓰립니다
```

:::